In [25]:
'''
CISC 453 Assignment 3
Windy Grid World
Harry Kim, Ryan D'gama
'''

import numpy as np 
import random 

def create_grid(): #creates size*size grid with the coordinates as values
    aList =[]
    for y in range(7):
        aList.append([10])
        for x in range(size):
            aList[y].append([])

    return aList
def choose_action(epsilon,s,Q,actions):
    #epsilon greedy
    if (random.uniform(0,1) > epsilon):
        #exploit
        a = find_best_action(Q,s)
        return a
    else:
        #explore
        a = random.randint(0,actions-1)
        return a

def find_best_action(Q,s):
    qlist =[]
    #add every q(s,a) for all a in s
    for action in Q:
        qlist.append(action[s[0],s[1]])
    #find the best a 
    a = np.argmax(qlist) 
    return a 

def take_action(s,a):
    next_s = s.copy()
    if a ==0:   #North
        next_s[0] -= 1
    elif a ==1:   #East
        next_s[1] += 1
    elif a ==2:   #South
        next_s[0] += 1
    elif a ==3:   #West
        next_s[1] -= 1
    elif a ==4:   #NorthEast
        next_s[0] -= 1
        next_s[1] += 1
    elif a ==5:   #SouthEast
        next_s[0] += 1
        next_s[1] += 1
    elif a ==6:   #SouthWest
        next_s[0] += 1
        next_s[1] -= 1
    elif a ==7:   #NorthWest
        next_s[0] -= 1
        next_s[1] -= 1

    return next_s

def out_of_bounds(s): 
    #returns boolean value of if the state is out of the grid
    if s[0] < 0 or s[0] > 6:
        return True
    if s[1] < 0 or s[1] > 9: 
        return True 
    else:
        return False

def apply_wind(next_s,s,wind):
    #you have to apply the original state's wind to the new state
    column = s[1]
    #make sure the column didn't go out of bounce, if so it just passes on to reward
    if column >= 0 and column <= 9:
        if next_s[0] >= 0 and next_s[0] <= 6:
            next_s[0] = next_s[0] - wind[column]
            #if wind pushes us out, it stays by the wall, reward doesn't punish us for it
            if next_s[0] < 0:
                next_s[0] = 0
    return next_s

def reward(og,coord,goal):
    #returns rewards and also makes sure the cell does not go out of bounds
    if out_of_bounds(coord) == True: #cell is out of bounds
        return [og,-5] #return to original state with -1 reward

    elif coord == goal: #reached the goal, reward = 10 
        return [coord,10]

    else:
        return [coord,-1] #if not move on with reward = 0

def get_path(Q,s,goal,wind):
    #prints out the path 
    total_moves = []
    while s != goal:
        total_moves.append(s)
        a = find_best_action(Q,s)
        
        print('\ns is ',s)
        next_s = take_action(s,a)
        print('S_next is',next_s)
        next_s = apply_wind(next_s,s,wind)
        print('S_next after wind is', next_s)
        next_s, r = reward(s,next_s,goal)
        s = next_s
        print("s is ",s, "a is",a,'next = ',next_s)
    print(len(total_moves))

def stochastic_wind():
    #creates stochastic winds with variation of 1 from the mean
    wind = [0,0,0,1,1,1,2,2,1,0]
    for i in range(len(wind)):
        if wind[i] != 0 :
            wind[i] = random.randint(wind[i]-1 , wind[i] +1)
    return wind

def sarsa(episodes,actions, epsilon, alpha, gamma, s_wind):
    #parameters
    epsilon = epsilon
    alpha = alpha
    gamma = gamma

    episodes = episodes 
    actions = actions #8 is kings moves, 4 is cardinal 
    start = [3,0]
    goal = [3,7]
    wind = [0,0,0,1,1,1,2,2,1,0]

    #Q -  8 of 7x10 arrays
    Q = np.zeros((actions,7,10))

    for j in range(episodes):

        s = start
        a = choose_action(epsilon,s,Q,actions)
                
        if s_wind == True:
            wind = stochastic_wind()

        while s != goal:
        #for i in range(1000):
            #take action A, observe s and r 
            next_s = take_action(s,a)
            #print('\n start loop')
            #print('S is ',s)
            #print('S_next is',next_s)
            next_s = apply_wind(next_s,s,wind)
            #print('S_next after wind is', next_s)
            next_s, r = reward(s,next_s,goal)
            #print('Final next_s is:',next_s)
            next_a = choose_action(epsilon,next_s,Q,actions)

            #update 
            #print("s is ",s, "a is",a, "r is",r, "next s is",next_s, "next a is ",next_a)
            Q[a][s[0]][s[1]] = Q[a][s[0]][s[1]] + alpha*(r + gamma*Q[next_a][next_s[0]][next_s[1]] - Q[a][s[0]][s[1]])       
            s = next_s
            a = next_a


    #print best path
    s = start
    #print(Q)
    get_path(Q,s,goal,wind)
    return Q

sar= sarsa(episodes = 100000, actions = 8, epsilon = 0.3, alpha = 0.001, gamma= 0.9, s_wind=False)


s is  [3, 0]
S_next is [3, 1]
S_next after wind is [3, 1]
s is  [3, 1] a is 1 next =  [3, 1]

s is  [3, 1]
S_next is [4, 2]
S_next after wind is [4, 2]
s is  [4, 2] a is 5 next =  [4, 2]

s is  [4, 2]
S_next is [5, 3]
S_next after wind is [5, 3]
s is  [5, 3] a is 5 next =  [5, 3]

s is  [5, 3]
S_next is [6, 4]
S_next after wind is [5, 4]
s is  [5, 4] a is 5 next =  [5, 4]

s is  [5, 4]
S_next is [5, 5]
S_next after wind is [4, 5]
s is  [4, 5] a is 1 next =  [4, 5]

s is  [4, 5]
S_next is [5, 6]
S_next after wind is [4, 6]
s is  [4, 6] a is 5 next =  [4, 6]

s is  [4, 6]
S_next is [5, 7]
S_next after wind is [3, 7]
s is  [3, 7] a is 5 next =  [3, 7]
7


In [19]:
def stochastic_wind():
    #creates stochastic winds with variation of 1 from the mean
    wind = [0,0,0,1,1,1,2,2,1,0]
    for i in range(len(wind)):
        if wind[i] != 0 :
            wind[i] = random.randint(wind[i]-1 , wind[i] +1)
    return wind
print(stochastic_wind())

[0, 0, 0, 1, 0, 1, 3, 3, 2, 0]


In [27]:
def Q_learning(episodes,actions, epsilon, alpha, gamma, s_wind = True):
    #parameters
    epsilon = 0.3
    alpha = 0.001
    gamma = 0.9

    episodes = episodes #8 is kings moves, 4 is cardinal 
    actions = actions
    start = [3,0]
    goal = [3,7]
    wind = [0,0,0,1,1,1,2,2,1,0]

    #Q -  8 of 7x10 arrays
    Q = np.zeros((actions,7,10))

    for _ in range(episodes):
        if s_wind == True:
            wind = stochastic_wind()
        s = start
        #print('yes we are still running')
        while s != goal:
            #take action A, observe s and r 
            a = choose_action(epsilon,s,Q,actions)
            next_s = take_action(s,a)
            #print('next_s before',next_s)
            next_s = apply_wind(next_s,s,wind)
            #print('next_s after',next_s)
            next_s, r = reward(s,next_s,goal)
            best_a = find_best_action(Q,next_s)

            #update 
            #print("s is ",s, "a is",a, "r is",r, "next s is",next_s)
            Q[a][s[0]][s[1]] = Q[a][s[0]][s[1]] + alpha*(r + gamma*Q[best_a][next_s[0]][next_s[1]] - Q[a][s[0]][s[1]])       
            s = next_s
    
    #print best path
    s = start

    get_path(Q,s,goal,wind)
    #print(Q)
    return Q
q_learn= Q_learning(episodes = 100000, actions = 8, epsilon = 0.1, alpha = 0.1, gamma= 0.9, s_wind = False)


s is  [3, 0]
S_next is [4, 1]
S_next after wind is [4, 1]
s is  [4, 1] a is 5 next =  [4, 1]

s is  [4, 1]
S_next is [5, 2]
S_next after wind is [5, 2]
s is  [5, 2] a is 5 next =  [5, 2]

s is  [5, 2]
S_next is [4, 3]
S_next after wind is [4, 3]
s is  [4, 3] a is 4 next =  [4, 3]

s is  [4, 3]
S_next is [5, 4]
S_next after wind is [4, 4]
s is  [4, 4] a is 5 next =  [4, 4]

s is  [4, 4]
S_next is [5, 5]
S_next after wind is [4, 5]
s is  [4, 5] a is 5 next =  [4, 5]

s is  [4, 5]
S_next is [5, 6]
S_next after wind is [4, 6]
s is  [4, 6] a is 5 next =  [4, 6]

s is  [4, 6]
S_next is [5, 7]
S_next after wind is [3, 7]
s is  [3, 7] a is 5 next =  [3, 7]
7


[3, 0]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
